In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, datetime, torch
from typing import *
from recstudio.utils import *
import torch.nn as nn

/home/mjyin/miniconda3/envs/recstudio/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
mask = torch.triu(torch.ones((5, 5), dtype=torch.bool), 1)

In [5]:
mask[:3] = 1

In [11]:
mask & (~torch.eye(5, dtype=torch.bool))

tensor([[False,  True,  True,  True,  True],
        [ True, False,  True,  True,  True],
        [ True,  True, False,  True,  True],
        [False, False, False, False,  True],
        [False, False, False, False, False]])

In [3]:
import recstudio.eval as eval
model, dataset = 'Test10', 'amazon-beauty'
model_class, model_conf = get_model(model)
model = model_class(model_conf)
dataset_class = model_class._get_dataset_class()

data_conf = {}

data_conf.update(model_conf['data'])    # update model-specified config
datasets = dataset_class(name=dataset, config=data_conf).build(**model_conf['data'])
train_data = datasets[0]


import logging
logger = get_logger('./test.log')
logger.setLevel(logging.ERROR)
model.logger = logging.getLogger('recstudio')

model._init_model(train_data)
model._init_parameter()
model.run_mode = 'light'
model.val_check = True
val_metrics = model.config['eval']['val_metrics']
model.val_metric = next(iter(val_metrics)) if isinstance(val_metrics, list) else val_metrics
cutoff = model.config['eval']['cutoff']
cutoffs = cutoff if isinstance(cutoff, list) else [cutoff]
if len(eval.get_rank_metrics(model.val_metric)) > 0:
    model.val_metric += '@' + str(cutoffs[0])
model.callback = model._get_callback(train_data.name)
model._accelerate()

In [4]:
save_dir = model.config['eval']['save_path']
model.save_dir = save_dir
ckpt = './saved/Test10/amazon-beauty/2023-05-17-11-10-47-028157.ckpt'
model.ckpt_path = './Test10/amazon-beauty/2023-05-17-11-10-47-028157.ckpt'
model.load_checkpoint(ckpt)
model.trainloaders = model._get_train_loaders(train_data)
model.optimizers = model._get_optimizers()

In [5]:
for g in model.optimizers[0]['optimizer'].param_groups:
    g['lr'] = 0.0001

In [6]:
model.finetune = True
model.query_encoder.finetune = False
for _ in range(100):
    out = model.evaluate(datasets[-1])
    print(out['ndcg@20'])
    print(out['recall@20'])

test
tensor(0.0403)
tensor(0.0933)
test
tensor(0.0407)
tensor(0.0936)
test
tensor(0.0408)
tensor(0.0936)
test
tensor(0.0405)
tensor(0.0935)
test
tensor(0.0405)
tensor(0.0931)
test
tensor(0.0403)
tensor(0.0934)
test
tensor(0.0404)
tensor(0.0933)
test
tensor(0.0407)
tensor(0.0935)
test
tensor(0.0406)
tensor(0.0934)
test
tensor(0.0408)
tensor(0.0935)
test
tensor(0.0405)
tensor(0.0937)
test
tensor(0.0405)
tensor(0.0936)
test
tensor(0.0407)
tensor(0.0936)
test
tensor(0.0406)
tensor(0.0938)
test
tensor(0.0406)
tensor(0.0931)
test
tensor(0.0406)
tensor(0.0934)
test
tensor(0.0406)
tensor(0.0935)
test
tensor(0.0404)
tensor(0.0934)
test
tensor(0.0403)
tensor(0.0935)
test
tensor(0.0405)
tensor(0.0936)
test
tensor(0.0404)
tensor(0.0930)
test
tensor(0.0405)
tensor(0.0934)
test
tensor(0.0402)
tensor(0.0932)
test
tensor(0.0406)
tensor(0.0935)
test
tensor(0.0404)
tensor(0.0935)
test
tensor(0.0406)
tensor(0.0930)
test
tensor(0.0405)
tensor(0.0928)
test
tensor(0.0404)
tensor(0.0928)
test
tensor(0.0403)
